# Project: Can we predict when, where, and which car is going to receive a parking ticket?

## Table of Contents:
* [Introduction](#1)
* [Wrangling](#2)
* [Exploratory Visuals](#3)
* [Explanatory Visuals](#4)
* [Conclusion](#5)

## Introduction:<a class="anchor" id="1"></a>
Have you ever browsed iMDB looking for good movies to watch, sorted by rating? Or browsing the movie you just watched on iMDB, only to find that it has a shockingly low or high viewer rating? What can we say about the high or low ratings of a movie on iMDB?

We take a dive into the dataset provided by Kaggle (but now replaced with TMDB ratings due to DMCA Takedown https://www.kaggle.com/tmdb/tmdb-movie-metadata/home) to see what's going on behind these scores!

In [4]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from patsy import dmatrices
import statsmodels.api as sm;
from datetime import datetime, timedelta
from statsmodels.stats.outliers_influence import variance_inflation_factor

%matplotlib inline

In [5]:
# Large dataset, we want to see all columns
pd.set_option('display.max_columns', None)

In [6]:
# read in .csv
df_og = pd.read_csv('parking-violations-issued-fiscal-year-2018.csv', low_memory = False)

## Wrangling:

### Data Issues
* violation code is type int -> cast as string

In [15]:
df_og.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Time First Observed,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Intersecting Street,Date First Observed,Law Section,Sub Division,Violation Legal Code,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,1105232165,GLS6001,NY,PAS,2018-07-03T00:00:00.000,14,SDN,HONDA,X,47130,13230,80030,20180702.0,78.0,78,968,86684,0968,0000,0811P,NaN,K,F,2,HANSON PLACE,NaN,0,408,D1,NaN,BBYBBBB,ALL,ALL,BLUE,0.0,2006,-,0,NaN,NaN,NaN,NaN,NaN
1,1121274900,HXM7361,NY,PAS,2018-06-28T00:00:00.000,46,SDN,NISSA,X,28990,14890,15040,20200203.0,112.0,112,968,103419,0968,0000,1145A,NaN,Q,F,71-30,AUSTIN ST,NaN,0,408,C,NaN,BBBBBBB,ALL,ALL,GRY,0.0,2017,-,0,NaN,NaN,NaN,NaN,NaN
2,1130964875,GTR7949,NY,PAS,2018-06-08T00:00:00.000,24,SUBN,JEEP,X,64,18510,99,20180930.0,122.0,122,835,0,0835,0000,0355P,NaN,R,NaN,NaN,GREAT KILLS BOAT LAU,NaN,0,408,D5,NaN,BBBBBBB,ALL,ALL,GREEN,0.0,0,-,0,NaN,NaN,NaN,NaN,NaN
3,1130964887,HH1842,NC,PAS,2018-06-07T00:00:00.000,24,P-U,FORD,X,11310,39800,39735,0.0,122.0,122,835,0,0835,0000,0123P,NaN,R,NaN,NaN,GREAT KILLS PARK BOA,NaN,0,408,D5,NaN,BBBBBBB,ALL,ALL,WHITE,0.0,0,-,0,NaN,NaN,NaN,NaN,NaN
4,1131599342,HDG7076,NY,PAS,2018-06-29T00:00:00.000,17,SUBN,HYUND,X,47130,13230,80030,20190124.0,78.0,78,868,2354,0868,0000,0514P,NaN,K,F,2,HANSON PLACE,NaN,0,408,C4,NaN,BBBBBBB,ALL,ALL,GREEN,0.0,2007,-,0,NaN,NaN,NaN,NaN,NaN


In [7]:
# let's create a copy we will work with
df = df_og.copy()

In [8]:
# Selecting the columns we want to work with
df = df[['Registration State', 'Issue Date', 'Violation Time', 'Violation Code', 'Street Name', 'Sub Division',
        'Vehicle Body Type', 'Vehicle Make', 'Vehicle Color', 'Vehicle Year']]

In [9]:
# change column names to lower case and eliminate empty space
df.columns = ['registration_state', 'issue_date', 'violation_time', 'violation_code', 'street_name', 
              'subdivision', 'vehicle_body_type', 'vehicle_make', 'vehicle_color', 'vehicle_year']

In [10]:
# drop duplicate rows and rows with missing value
df.drop_duplicates(inplace = True)
df.dropna(inplace = True)

In [12]:
# Convert Issue Date to datetime object
df.issue_date = pd.to_datetime(df.issue_date)

In [13]:
# Get rid of incorrect violation time data
mask = (df.violation_time.str.len() == 5) & (df.violation_time.str.count('\.') == 0) & (df.violation_time.str.count(' ') == 0)

df = df.loc[mask]

In [14]:
# Add flag column for additional filtering later on
df['flag'] = False
df = df.reset_index(drop=True)

In [17]:
for i in df.index:
    # grab violation time at row i
    v_time = df.get_value(i, 'violation_time')
    
    # tokenize hour, minute, and PM/AM characters into different variables, cast to int as necessary
    h = int(v_time[0:2])
    m = int(v_time[2:4])
    p = v_time[4:5]
    
    # flag rows with erroneous violation time values (ex. 6831P) 
    if h >= 24:
        df.set_value(i, 'flag', True)   
    if m >= 60:
        df.set_value(i, 'flag', True)
    
    # if violation time is PM, add 12 to hour value
    if p == 'P':
        h += 12
    
    # add time data to issue date data using timedelta
    df.set_value(i, 'issue_date', df.issue_date[i] + timedelta(hours = h, minutes = m))

A:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
A:\Anaconda3\lib\site-packages\ipykernel\__main__.py:21: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
A:\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [18]:
# remove flagged rows
df = df.query('flag == False')

In [19]:
# drop violation time as it's no longer needed
df.drop('violation_time', axis = 1, inplace = True)

In [20]:
df.head()

,registration_state,issue_date,violation_code,street_name,subdivision,vehicle_body_type,vehicle_make,vehicle_color,vehicle_year,flag
0,NY,2018-07-03 20:11:00,14,HANSON PLACE,D1,SDN,HONDA,BLUE,2006,False
1,NY,2018-06-28 11:45:00,46,AUSTIN ST,C,SDN,NISSA,GRY,2017,False
2,NY,2018-06-08 15:55:00,24,GREAT KILLS BOAT LAU,D5,SUBN,JEEP,GREEN,0,False
3,NC,2018-06-07 13:23:00,24,GREAT KILLS PARK BOA,D5,P-U,FORD,WHITE,0,False
4,NY,2018-06-29 17:14:00,17,HANSON PLACE,C4,SUBN,HYUND,GREEN,2007,False


In [34]:
# create a dictionary for all registration states of North America (US, Canada, Mexico)
states = {'AL', 'AK', 'AZ', 'AR', 'AA', 'AE', 'AP', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA',
         'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND',
         'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'AB', 'BC', 'MB',
         'NB', 'NF', 'NT', 'NS', 'NU', 'ON', 'PE', 'QC', 'SK', 'YT', 'AG', 'BJ', 'BS', 'CP', 'CH', 'CI', 'CU', 'CL', 'DF',
         'DG', 'GJ', 'GR', 'HG', 'JA', 'EM', 'MH', 'MR', 'NA', 'NL', 'OA', 'PU', 'QA', 'QR', 'SL', 'SI', 'SO', 'TA', 'TM',
         'TL', 'VZ', 'YC', 'ZT'}

In [38]:
# if registration state code is not found in states dictionary, flag it as True
df.flag = ~df.registration_state.isin(states)

In [43]:
# cast violation code as string object, we will not be performing numeric calculation with these nominal values
df.violation_code = df.violation_code.astype('str')

In [64]:
df.to_csv('temp.csv', index = False)

In [74]:
# remove all parking violations that had less than mean amount (142) of all parking violation locations
#s = df.street_name.value_counts()
#df = df[df.isin(s.index[s >= 142]).values]

In [80]:
df = pd.read_csv('temp.csv')

In [81]:
df.head()

,registration_state,issue_date,violation_code,street_name,subdivision,vehicle_body_type,vehicle_make,vehicle_color,vehicle_year,flag
0,NY,2018-07-03 20:11:00,14,HANSON PLACE,D1,SDN,HONDA,BLUE,2006,False
1,NY,2018-06-28 11:45:00,46,AUSTIN ST,C,SDN,NISSA,GRY,2017,False
2,NY,2018-06-08 15:55:00,24,GREAT KILLS BOAT LAU,D5,SUBN,JEEP,GREEN,0,False
3,NC,2018-06-07 13:23:00,24,GREAT KILLS PARK BOA,D5,P-U,FORD,WHITE,0,False
4,NY,2018-06-29 17:14:00,17,HANSON PLACE,C4,SUBN,HYUND,GREEN,2007,False


There is a huge amount of mess here with **vehicle body types**. Same type of vehicles are represented with different abbreviations, and some don't even appear under official NY vehicle registration sheet (https://data.ny.gov/api/assets/83055271-29A6-4ED4-9374-E159F30DB5AE). I will be combining the same vehicle types under official abbreviations. For the purposes of this analysis we will only be concerned with commercially distinguishable vehicle body types.

SUBN    
4DSD + 4 DR + 4D  
VAN    
DELV  
PICK + P-U + PKUP  
2DSD  
REFG   
TRAC  
UTIL + UT    
TAXI        
CONV   
BUS         
TRLR + SEMI    
MCY      
FLAT + TRK + TOW  
DUMP       
TR/C  

There is also a huge amount of mess with **vehicle make**. Same type of vehicles are represented with different abbv, and may do not appear under NY's official vehicle make database (https://data.ny.gov/Transportation/Vehicle-Makes-and-Body-Types-Most-Popular-in-New-Y/3pxy-wy2i). For the purposes of the analysis, commercially distinguishable vehicle makes will be combined under 1 abbreviation.

In [91]:
#df.vehicle_make.value_counts().index.tolist()

Some mess here in the **vehicle year** data. Around 1.3 car's manufacture year could not be identified. This dataset collects all parking enforcement in NY in 2018 and 2019 car models do go on sale in 2018. Therefore, we will be getting rid of all rows with manufacture date recorded as 2020 and up.